In [1]:
import time
from IPython.display import display
from ipywidgets import widgets
from ipywidgets import interact, Button, Output, HBox, VBox
from threading import Thread
import asyncio
from trademanager import run_system
from plotting import plot_backtest_charts

param = {}
global candles_dict
candles_dict = {}

#--------------------------Paramters-----------------------------------------------------------


''' available capital for trade in USD '''
param['account_capital'] = 10000

''' 
Portfolio: ma_breakout_strategy, mean_reversion_strategy 
trade_capital: percent of account_capital
param['portfolio_dict'] = {'ma_breakout_strategy':{'AAPL':{'trade_capital':10},'AMZN':{'trade_capital':40}}, 
                           'mean_reversion_strategy':{'AAPL':{'trade_capital':10},'TSLA':{'trade_capital':10}}}
'''

param['portfolio_dict'] = {
                            'chori_strategy':
                            {
                                #'AAPL':{'trade_capital':10},
                                #'AMZN':{'trade_capital':40},
                                #'SPY':{'trade_capital':10},   
                            },
                            'bros24_strategy':
                            {
                                'AAPL':{'trade_capital':10},
                                #'AMZN':{'trade_capital':40},
                                #'SPY':{'trade_capital':10},   
                            }
                          }


param['indicator_dict'] = {
                            'chori_strategy':
                            {
                                'AAPL':[('ChoriSMA','line'),('ChoriEMA','line'), \
                                        ('chori_strategy'+'stoplossprice','dash'),\
                                        ('chori_strategy'+'takeprofitprice','dash')],
                                'AMZN':[('ChoriSMA','line'),('ChoriEMA','line'), \
                                        ('chori_strategy'+'stoplossprice','dash'),\
                                        ('chori_strategy'+'takeprofitprice','dash')],
                                'SPY':[('ChoriSMA','line'),('ChoriEMA','line'), \
                                        ('chori_strategy'+'stoplossprice','dash'),\
                                        ('chori_strategy'+'takeprofitprice','dash')],   
                            },
                            'bros24_strategy':
                            {
                                'AAPL':[('bros24_i','line'),('bros24_i2','line'),\
                                        ('bros24_s','line'),('bros24_s2','line'),\
                                        ('bros24_strategy'+'takeprofitprice','dash'),\
                                        ('bros24_strategy'+'stoplossprice','dash')],
                                'AMZN':[('bros24_i','line'),('bros24_i2','line'),\
                                        ('bros24_s','line'),('bros24_s2','line')],
                                'SPY':[('bros24_i','line'),('bros24_i2','line'),\
                                       ('bros24_s','line'),('bros24_s2','line')],   
                            },
                           
                          }


#---------------------- system settings--------------------------------------------

'''
Direction of trade, in options, it's LONG->CALL option, SHORT->PUT option, 
BOTH->CALL and PUT option,  'LONG', 'SHORT', 'BOTH'
'''  

param['tradeDirection'] = 'BOTH'
#interact(f, x=['apples','oranges']);


'''
Chori strategy parameters
'''

param['ChoriSMAPeriod'] = 1
param['ChoriEMAPeriod'] = 3
param['steppingladderexit'] = True


''' Trading window for placing orders '''
param['useTradewindow'] = True
param['tradewindowStart'] = 935
param['tradewindowEnd'] = 1550


''' Intra-day squaring of positions'''
param['tradeSquare'] = True
param['tradeSquareTime'] = 1555


''' Strategy: ma_breakout_strategy, mean_reversion_strategy '''
''' 
Check for low volatility, 
default value in absence of symbol is True  
'''
param['useVolatility_dict'] = {"SPY":True,"TSLA":False,'AAPL':True,'AMZN':False}


param['strategy'] = 'ma_breakout_strategy'

param['mean_reversion_period'] = 10
param['mean_reversion_secondary_period'] = 100
param['z_entry'] = 0.9
param['z_exit'] = 0.5


''' EMA crossing over/under SMA '''
param['useEMA'] = True
param['EMAPeriod'] = 12

param['useSMA'] = True
param['SMAPeriod'] = 20



'''spread constraint'''
param['maxOptionSpreadPercent'] = 1.0


'''
default lot size of each symbol, actual size will be 
calculated using trade_capital in portfolio_dict
'''
param['LOT_dict'] = {"SPY":1,"GOOG":1,"MED":1,"TSLA":1,'AAPL':1,'AMZN':1}

'''
RUN_FOR_MINUTES: Duration for loop (minutes), this value
should be set as 0 for backtesting on security
'''

param['RUN_FOR_MINUTES'] = 0


'BE CAREFUL! It will place live orders in your trade account (False/True)'
param['PLACE_ORDER'] = False


'''
Following instrument list contains symbol to be traded
input is list like param['instrument_symbol_list'] =  ["SPY","TSLA","AAPL", "AMZN","GOOG","MED"]
'''

param['instrument_symbol_list'] = []

#-------------------------trader input settings --------------------------------


'update intra-bar tick data every N seconds'
param['UPDATE_TICK_DATA_SECONDS'] = 1


'paper trading in live market'
param['TRADE_LIVE'] = True


'allow short/put options'
param['ALLOW_SHORT'] = True


'generate refresh token every 90 days by switching it to True and then turn False again'
param['generate_refresh_token'] = False


#--------------------- time frame and history data paramters ---------------
param['timeframe']='minute'
param['frequency'] = '1'
param['periodType']='day'
param['period']='7'
param['extended_hours'] = False


#---------------------------------post process------------------------------
param['liveOutputFrequency'] = 5

print('param complete')

param complete


In [2]:
button = widgets.Button(description="Run System!")
button.style.button_color = 'lightgreen'
output = widgets.Output()
display(button, output)
def on_button_clicked_thread(b):
    print('running thread')
    thread = Thread(target=on_button_clicked,args=(b,))
    thread.start()
    thread.join()
def on_button_clicked(b):
    global candles_dict
    with output:
        candles_dict = run_system(param, None)
        
button.on_click(on_button_clicked_thread)


Button(description='Run System!', style=ButtonStyle(button_color='lightgreen'))

Output()

running thread


In [3]:
global fig_dict
fig_dict = {}                   
chart = widgets.Button(description="Backtest Chart")
chart.style.button_color = 'lightblue'
chart_output = widgets.Output()
display(chart, chart_output)
def on_charting(b):
    global fig_dict
    with chart_output:
        if True:
            for portfolio in param['portfolio_dict']:
                symbols = param['portfolio_dict'][portfolio]
                for symbol in symbols.keys():
                    indicator_list = param['indicator_dict'][portfolio][symbol]
                    fig_dict[portfolio+symbol] = plot_backtest_charts(portfolio=portfolio, symbols=[symbol], \
                                 timeframe=param['timeframe'], frequency=param['frequency'], \
                                 periodType=param['periodType'], period=param['period'], \
                                 params=None, tradeDirection=param['tradeDirection'], 
                                 extended_hours=False, candles_dict=candles_dict, indicator_list=indicator_list)
        else:
            print('failed')
            return
chart.on_click(on_charting)

Button(description='Backtest Chart', style=ButtonStyle(button_color='lightblue'))

Output()